In [17]:
#1. rutin1 import module
import pandas as pd
import os
import sys

In [18]:
#2. rutin2 membuat syspath ke root utk aktifkan __init__.py
repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
sys.path.append(repo_root)

In [19]:
#3. rutin3 Load the dataset
dataset_path = os.path.join(repo_root, "data/dataset_ekstraksi_r30_lg.xlsx")
df = pd.read_excel(dataset_path)
df["text"] = df["judul"] +". "+ df["isi"]

In [20]:
# Preprocess
import re

def preprocess(text):
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    text = text.replace('.', '. ')
    text = re.sub('[^a-zA-Z.]', ' ', text)
    text = text.lower()
    text = re.sub("(\\d|\\W)+"," ",text)
    text = text.strip()
    text = text.split()

    return text

df_preprocess = df['text'].apply(preprocess)
df_preprocess.head(3)

0    [usulan, personil, penting, proposed, key, per...
1    [template, document, jtb, gpf, project, mengac...
2    [change, inquiry, terkait, usulan, perubahan, ...
Name: text, dtype: object

In [21]:
from gensim.models import Word2Vec

# Combine the tokenized text from 'judul' and 'isi' columns for Word2Vec training
combined_text = df_preprocess.tolist()

# Train the Word2Vec model
model = Word2Vec(sentences=combined_text, vector_size=200, window=5, min_count=1, workers=4)

In [22]:
# Show the vector for a sample word to verify the model has been trained
sample_word = "personil"
if sample_word in model.wv:
    sample_vector = model.wv[sample_word]
    sample_vector[:10]  # Show the first 10 dimensions of the vector
else:
    sample_vector = "The word is not in the model's vocabulary."
sample_vector


array([ 0.3805954 ,  0.19502722, -0.04969674, -0.052543  ,  0.6276642 ,
       -0.26888612,  0.34123692,  0.6729766 , -0.48455048, -0.07949165,
       -0.46155065, -0.5681077 , -0.08227619,  0.21913697, -0.13342686,
        0.01275626,  0.04953588, -0.02091675, -0.24995239, -0.7832174 ,
       -0.0140854 , -0.15341513,  0.43259844, -0.21324211, -0.08191957,
       -0.2850188 , -0.01729772,  0.00750273, -0.24520853, -0.02684724,
        0.5310569 ,  0.5091917 ,  0.25019056, -0.16600998,  0.00123054,
        0.07833659,  0.23094776,  0.13595912,  0.13039918, -0.39653382,
       -0.2139622 ,  0.0718846 , -0.04618865, -0.24046537,  0.20447627,
       -0.2740968 ,  0.02208405,  0.13364516,  0.2741228 ,  0.1346074 ,
        0.21759163,  0.02268464,  0.11243378, -0.53286135, -0.38485318,
       -0.24340096, -0.12109352, -0.4352853 , -0.39904583,  0.07851148,
        0.15959558, -0.38711897,  0.05793848,  0.09909549, -0.6065178 ,
        0.3283644 , -0.00252021,  0.6984353 , -0.63422984,  0.18

In [23]:
model.save("word2vec_model_surat_lg.model")
